In [ ]:
from time import time

start_time = time()

df = spark.read.format("delta").load("/mnt/datalake/bronze/transacoes")
df.filter(col("ano") == 2025).count()

end_time = time()
print(f"Tempo da query antes da otimização: {end_time - start_time:.2f} segundos")






In [ ]:
df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("ano", "mes") \
    .save("/mnt/datalake/silver/transacoes")

In [ ]:
OPTIMIZE delta.`/mnt/datalake/silver/transacoes` ZORDER BY (customer_id);

In [ ]:
ALTER TABLE delta.`/mnt/datalake/silver/transacoes`
ADD CONSTRAINT bloomfilter_customer_id
USING BLOOMFILTER ON (customer_id);

In [ ]:
start_time = time()

df_optimized = spark.read.format("delta").load("/mnt/datalake/silver/transacoes")
df_optimized.filter(col("ano") == 2023).count()

end_time = time()
print(f"Tempo da query após a otimização: {end_time - start_time:.2f} segundos")
